# 0. Importing Relevant Libraries

In [9]:
# !pip install kagglehub
# http://kaggle.com/datasets/pabloramoswilkins/ucl-2025-players-data/data

import os

# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("pabloramoswilkins/ucl-2025-players-data")

# print("Path to dataset files:", path)

# print(os.getcwd())

import pandas as pd
from functools import reduce
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, RFE, SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# 1. Reading and Cleaning the Data

In [26]:
df_attacking = pd.read_csv("data/attacking_data.csv")
df_attempts = pd.read_csv("data/attempts_data.csv")

df_defending = pd.read_csv("data/defending_data.csv")
df_disciplinary = pd.read_csv("data/disciplinary_data.csv")
df_distribution = pd.read_csv("data/distribution_data.csv")

df_goalkeeping = pd.read_csv("data/goalkeeping_data.csv")
df_goals = pd.read_csv("data/goals_data.csv")

df_key_stats = pd.read_csv("data/key_stats_data.csv")
df_players = pd.read_csv("data/players_data.csv")

df_teams = pd.read_csv("data/teams_data.csv")
# df_goalkeeping
df_disciplinary

,Unnamed: 0,id_player,fouls_committed,fouls_suffered,yellow_cards,red_cards
0,0,250112690,11.0,7.0,1,0
1,1,250065419,11.0,4.0,2,0
2,2,250105927,10.0,7.0,0,0
3,3,250129965,10.0,5.0,1,0
4,4,250025791,9.0,7.0,2,0
...,...,...,...,...,...,...
903,903,250011071,NaN,NaN,0,0
904,904,250000104,NaN,NaN,0,0
905,905,108501,NaN,NaN,0,0
906,906,103827,NaN,NaN,0,0


In [11]:
df_attacker_valuations = pd.read_csv("data_valuations/attacker_valuations.csv")
df_defender_valuation = pd.read_csv("data_valuations/defender_valuations.csv")
df_defender_goalkeeper = pd.read_csv("data_valuations/goalkeeper_valuations.csv")
df_defender_midfielder = pd.read_csv("data_valuations/midfielder_valuations.csv")
df_attacker_valuations
df_defender_midfielder

df_all_valuations = pd.concat([
    df_attacker_valuations,
    df_defender_valuation,
    df_defender_goalkeeper,
    df_defender_midfielder
], ignore_index=True)

# df_all_valuations

In [27]:
dfs = [df_defending, df_distribution, df_goalkeeping, df_disciplinary, df_players]

df_merged = reduce(lambda left, right: pd.merge(left, right, on='id_player', how='left'), dfs)
df_merged_valuations = pd.merge(df_merged, df_all_valuations, left_on='player_name', right_on='Name', how='left')
df_merged_valuations_goalkeeper = df_merged_valuations[df_merged_valuations["Position"] == "Goalkeeper"]


In [55]:
dfs = [df_defending, df_distribution, df_goalkeeping, df_disciplinary, df_players]

df_merged = reduce(lambda left, right: pd.merge(left, right, on='id_player', how='left'), dfs)
df_merged_valuations = pd.merge(df_merged, df_all_valuations, left_on='player_name', right_on='Name', how='left')
df_merged_valuations_goalkeeper = df_merged_valuations[df_merged_valuations["Position"] == "Goalkeeper"]
# df_merged_valuations

# df_merged_valuations_goalkeeper.drop_duplicates()
# df_merged_valuations_goalkeeper.dropna().drop_duplicates()

df_merged_valuations_goalkeeper.dtypes

def convert_to_integer(value):
    value = value.replace('€', '').replace('m', '').replace('k', '')  # Remove currency symbol and 'm'
    return int(float(value) * 1_000_000)  # Convert to float, then multiply by 1 million

df_mv = df_merged_valuations_goalkeeper["Market Value"].apply(convert_to_integer)
df_merged_valuations_goalkeeper["Market Value"] = df_mv
df_merged_valuations_goalkeeper.head(4)

/var/folders/yl/h0sn7vdn6pg66cvg_h6mc02h0000gn/T/ipykernel_11222/1904265595.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_valuations_goalkeeper["Market Value"] = df_mv


,id_player,balls_recovered,tackles,tackles_won,tackles_lost,clearance_attempted,passing_accuracy(%),passes_attempted,passes_completed,crossing_accuracy(%),...,weight(kg),height(cm),age,id_team,player_image,Name,Position,Age,Market Value,Club Name
600,250090955,0.0,1.0,1.0,0.0,1.0,86.00,39.0,33.0,0.0,...,NaN,NaN,26,50023,https://img.uefa.com/imgml/TP/players/1/2025/3...,Philipp Köhn,Goalkeeper,26.0,4000000,AS Monaco
611,250052744,0.0,1.0,0.0,1.0,1.0,76.00,63.0,47.0,0.0,...,NaN,192.0,29,50031,https://img.uefa.com/imgml/TP/players/1/2025/3...,David von Ballmoos,Goalkeeper,30.0,1200000,BSC Young Boys
625,250135068,0.0,0.0,0.0,0.0,8.0,65.00,142.0,93.0,0.0,...,NaN,NaN,23,75797,https://img.uefa.com/imgml/TP/players/1/2025/3...,Lucas Chevalier,Goalkeeper,23.0,30000000,LOSC Lille
626,250069832,0.0,0.0,0.0,0.0,7.0,88.34,112.0,100.0,0.0,...,NaN,NaN,26,52758,https://img.uefa.com/imgml/TP/players/1/2025/3...,Gregor Kobel,Goalkeeper,27.0,40000000,Borussia Dortmund


In [56]:
df_merged_valuations_goalkeeper.dtypes

id_player                 int64
balls_recovered         float64
tackles                 float64
tackles_won             float64
tackles_lost            float64
clearance_attempted     float64
passing_accuracy(%)     float64
passes_attempted        float64
passes_completed        float64
crossing_accuracy(%)    float64
crosses_attempted       float64
crosses_completed       float64
free_kick_taken         float64
matches_appearance      float64
saves                   float64
goals_conceded          float64
saves_on_penalty        float64
clean_sheets            float64
punches_made            float64
Unnamed: 0                int64
fouls_committed         float64
fouls_suffered          float64
yellow_cards              int64
red_cards                 int64
player_name              object
nationality              object
field_position           object
position                 object
weight(kg)              float64
height(cm)              float64
age                       int64
id_team 

# 2. Simple Linear Regression

##

In [ ]:
Variable_list = ["clean_sheets", "saves", "clearance_attempted", "Age", "Market Value","passes_completed", "punches_made", "fouls_committed", "yellow_cards",
                 "red_cards", "matches_appearance", "saves_on_penalty"]

y = df_merged_valuations_goalkeeper[Variable_list].dropna().iloc[:, 0]
X = df_merged_valuations_goalkeeper[Variable_list].dropna().iloc[:, 1:]

In [59]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### 1️⃣ FILTER METHOD: SelectKBest (Chi-Square)
print("\n--- Filter Method (Chi-Square) ---")
chi_selector = SelectKBest(score_func=chi2, k=2)
X_kbest = chi_selector.fit_transform(X_train, y_train)
selected_features = X_train.columns[chi_selector.get_support()]
print(f"Selected Features: {list(selected_features)}")

### 2️⃣ WRAPPER METHOD: Recursive Feature Elimination (RFE)
print("\n--- Wrapper Method (RFE) ---")
model = LogisticRegression(max_iter=500)
rfe = RFE(estimator=model, n_features_to_select=2)
rfe.fit(X_train, y_train)
selected_features = X_train.columns[rfe.support_]
print(f"Selected Features: {list(selected_features)}")

### 3️⃣ EMBEDDED METHOD: Feature Importance (Random Forest)
print("\n--- Embedded Method (Random Forest Feature Importance) ---")
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
importances = model.feature_importances_

# Create a DataFrame for feature importance
feat_importances = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances})
feat_importances = feat_importances.sort_values(by='Importance', ascending=False)
print(feat_importances)

### OPTIONAL: Embedded Method (Lasso Regularization)
from sklearn.linear_model import Lasso
print("\n--- Embedded Method (Lasso) ---")
lasso = Lasso(alpha=0.01)
lasso.fit(X_train, y_train)
lasso_features = X_train.columns[lasso.coef_ != 0]
print(f"Selected Features by Lasso: {list(lasso_features)}")



--- Filter Method (Chi-Square) ---
Selected Features: ['Market Value', 'passes_completed']

--- Wrapper Method (RFE) ---
Selected Features: ['Market Value', 'passes_completed']

--- Embedded Method (Random Forest Feature Importance) ---
                Feature  Importance
4      passes_completed    0.204926
3          Market Value    0.159960
0                 saves    0.134749
1   clearance_attempted    0.115244
2                   Age    0.113103
9    matches_appearance    0.112638
5          punches_made    0.107787
10     saves_on_penalty    0.022794
6       fouls_committed    0.014847
7          yellow_cards    0.011567
8             red_cards    0.002386

--- Embedded Method (Lasso) ---
Selected Features by Lasso: ['saves', 'clearance_attempted', 'Age', 'Market Value', 'passes_completed', 'punches_made', 'fouls_committed', 'yellow_cards', 'matches_appearance']


In [ ]:
# pick the top features

# Variable_list = ["clean_sheets", "saves", "clearance_attempted", "Age", "Market Value","passes_completed", "punches_made", "fouls_committed", "yellow_cards",
#                  "red_cards", "matches_appearance", "saves_on_penalty"]

Variable_list = ["clean_sheets","Market Value","passes_completed", "clearance_attempted"]


y = df_merged_valuations_goalkeeper[Variable_list].dropna().iloc[:, 0]
X = df_merged_valuations_goalkeeper[Variable_list].dropna().iloc[:, 1:]

In [63]:
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [65]:
X = sm.add_constant(X)  # This is required for statsmodels
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           clean_sheets   R-squared:                       0.219
Model:                            OLS   Adj. R-squared:                  0.162
Method:                 Least Squares   F-statistic:                     3.829
Date:                Tue, 18 Feb 2025   Prob (F-statistic):             0.0166
Time:                        22:28:39   Log-Likelihood:                -59.892
No. Observations:                  45   AIC:                             127.8
Df Residuals:                      41   BIC:                             135.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -0.1735      0.329     -0.527      0.601      -0.839       0.492
Market Value         2.361e-10   8.09e-10      0.292      0.772    -1.4e-09    1.87e-09
passes_completed        0.0103      0.004      2.568      0.014       0.002       0.018
clearance_attempted     0.0750      0.084      0.888      0.380      -0.096       0.246
==============================================================================
Omnibus:                       19.484   Durbin-Watson:                   1.979
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               26.398
Skew:                           1.424   Prob(JB):                     1.85e-06
Kurtosis:                       5.444   Cond. No.                     4.55e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.55e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# 3. Random Forest

In [66]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Determine if classification or regression based on the type of target values
if y.nunique() > 10 and y.dtype in ['float64', 'int64']:
    # Regression Model
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    print("Model Type: Regression (Random Forest)")
    print(f"Mean Squared Error: {mse}")
    print(f"R-squared: {r2}")

else:
    # Classification Model
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    class_report = classification_report(y_test, predictions)

    print("Model Type: Classification (Random Forest)")
    print(f"Accuracy: {accuracy}")
    print("\nClassification Report:")
    print(class_report)


Model Type: Classification (Random Forest)
Accuracy: 0.5

Classification Report:
              precision    recall  f1-score   support

         0.0       0.75      0.75      0.75         8
         1.0       0.50      0.20      0.29         5
         2.0       0.00      0.00      0.00         0
         4.0       0.00      0.00      0.00         1

    accuracy                           0.50        14
   macro avg       0.31      0.24      0.26        14
weighted avg       0.61      0.50      0.53        14



/Users/stefanospillo/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/stefanospillo/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/stefanospillo/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this